This notebook is just to use `timeit` to time small functions and figure out what are the slowest in the package. It is not for demonstrative purposes.

In [1]:
import frispy
import timeit

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np

In [4]:
def do_timeit(fname, number=100):
    setup = f"from __main__ import {fname}"
    t = timeit.timeit(f"{fname}()", number=number, setup=setup)
    return f"{fname}: {number} iterations took {t:.4f} seconds; {t/number:.2e} s/iter"

In [5]:
trajectory = frispy.Trajectory()
phi = np.pi / 2.
theta = np.pi / 4.
vel = np.random.rand(3)
w = np.random.rand(3)

def calc_intermediate_quantities():
    _ = trajectory.calculate_intermediate_quantities(phi, theta, vel, w)

In [6]:
def calc_R():
    _ = trajectory.rotation_matrix(0.12324, 0.9874)

print(do_timeit("calc_R", 626))

calc_R: 626 iterations took 0.0063 seconds; 1.01e-05 s/iter


In [7]:
print(do_timeit("calc_intermediate_quantities", 626))

calc_intermediate_quantities: 626 iterations took 0.0709 seconds; 1.13e-04 s/iter


In [8]:
eom = frispy.EOM()
res = eom.compute_forces(phi, theta, vel, w)

def compute_forces():
    _ = eom.compute_forces(phi, theta, vel, w)

def compute_torques():
    _ = eom.compute_torques(vel, w, res)

In [9]:
print(do_timeit("compute_forces", 626))

compute_forces: 626 iterations took 0.1260 seconds; 2.01e-04 s/iter


In [10]:
print(do_timeit("compute_torques", 626))

compute_torques: 626 iterations took 0.0169 seconds; 2.70e-05 s/iter


In [11]:
d = frispy.Disc()
coords = d.initial_conditions_as_ordered_list
d.initial_conditions["vx"] = 40

def compute_derivatives():
    _ = eom.compute_derivatives(0, np.array(coords))

def compute_trajectory():
    _ = d.compute_trajectory()

In [12]:
print(do_timeit("compute_derivatives", 626))

compute_derivatives: 626 iterations took 0.1963 seconds; 3.14e-04 s/iter


In [13]:
print(do_timeit("compute_trajectory", 10))

compute_trajectory: 10 iterations took 43.4771 seconds; 4.35e+00 s/iter


In [14]:
from scipy.integrate import solve_ivp, odeint

def call_solver():
    _ = solve_ivp(
        fun=d.eom.compute_derivatives,
        t_span=(0, 3),
        y0=d.initial_conditions_as_ordered_list,
    )

In [15]:
print(do_timeit("call_solver", 10))

call_solver: 10 iterations took 43.4658 seconds; 4.35e+00 s/iter


In [16]:
y0 = d.initial_conditions_as_ordered_list
# y0[3] = 13.
# y0[0] = 0.
print(y0)

[0, 0, 1.0, 40, 0, 0, 0, 0, 0, 0, 0, 62.0]


In [17]:
r = solve_ivp(
    fun=d.eom.compute_derivatives,
    t_span=(0, 3),
    y0=y0,
)

In [18]:
y0[0] = 0
y0[3] = 13.
def call_solver2():
    _ = solve_ivp(
        fun=d.eom.compute_derivatives,
        t_span=(0, 3),
        y0=y0,
#        first_step=0.05,
        rtol=2e-2,
        atol=5e-3,
#        method="LSODA"
    )

In [19]:
print(do_timeit("call_solver2", 100))

call_solver2: 100 iterations took 3.8866 seconds; 3.89e-02 s/iter


In [20]:
t = np.linspace(0, 3, 1000)

def compute_derivs_static(coords, time, disc):
    return disc.eom.compute_derivatives(time, coords)

def call_odeint():
    _ = odeint(compute_derivs_static, t=t, y0=y0, args=(d,))

In [21]:
print(do_timeit("call_odeint", 10))

call_odeint: 10 iterations took 2.6520 seconds; 2.65e-01 s/iter
